In [1]:
# !pip install torchsummary

In [2]:
import torch
import os
import cv2
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.torch_version
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
# from torchsummary import summary
from torchvision import transforms
from torchmetrics import Accuracy, Precision, Recall
import timm
import cv2

from torch.cuda.amp import GradScaler, autocast


In [3]:
import torch
print(torch.__version__)

2.1.2


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
main_path = '/kaggle/input/isic-2024-challenge' if iskaggle else 'data/isic-2024-challenge'

In [6]:
train_metadata_path = '/kaggle/input/isic-2024-challenge/train-metadata.csv' if iskaggle else 'data/isic-2024-challenge/train-metadata.csv'
test_metadata_path = '/kaggle/input/isic-2024-challenge/test-metadata.csv' if iskaggle else 'data/isic-2024-challenge/test-metadata.csv'

train_metadata_df = pd.read_csv(train_metadata_path)
test_metadata_df = pd.read_csv(test_metadata_path)

print(len(train_metadata_df))

/tmp/ipykernel_24/3275392473.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata_df = pd.read_csv(train_metadata_path)


401059


In [7]:
import sklearn.model_selection as train_test_split

train_size = 0.8
# Splitting the train dataset into positive and negative samples and saving them in separate dataframes
postive_samples = train_metadata_df[train_metadata_df['target'] == 1]
negative_samples = train_metadata_df[train_metadata_df['target'] == 0]

# postive_samples = postive_samples.sample(frac=0.1)
negative_samples = negative_samples.sample(frac=100*(postive_samples.shape[0]/negative_samples.shape[0]))

print(f"Positive samples: {postive_samples.shape}")
print(f"Negative samples: {negative_samples.shape}")

Positive samples: (393, 55)
Negative samples: (39300, 55)


In [8]:

# Splitting each type of samples into train and validation sets
train_positive_samples, val_positive_samples = train_test_split.train_test_split(postive_samples,test_size=1-train_size)
train_negative_samples, val_negative_samples = train_test_split.train_test_split(negative_samples,test_size=1-train_size)
print(f"Train positive samples: {train_positive_samples.shape}")
print(f"Train negative samples: {train_negative_samples.shape}")
print(f"Val positive samples: {val_positive_samples.shape}")
print(f"Val negative samples: {val_negative_samples.shape}")


Train positive samples: (314, 55)
Train negative samples: (31440, 55)
Val positive samples: (79, 55)
Val negative samples: (7860, 55)


In [9]:

# Concatenating the positive and negative samples to get the train and validation sets
train_metadata_df = pd.concat([train_positive_samples, train_negative_samples])
val_metadata_df = pd.concat([val_positive_samples, val_negative_samples])
print(f"Train samples: {train_metadata_df.shape}")
print(f"Val samples: {val_metadata_df.shape}")
print(f"Test samples: {test_metadata_df.shape}")


Train samples: (31754, 55)
Val samples: (7939, 55)
Test samples: (3, 44)


In [10]:
import h5py
from io import BytesIO

train_hdf5_path = '/kaggle/input/isic-2024-challenge/train-image.hdf5' if iskaggle else 'data/isic-2024-challenge/train-image.hdf5'
test_hdf5_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5' if iskaggle else 'data/isic-2024-challenge/test-image.hdf5'
train_image_path = '/kaggle/input/isic-2024-challenge/train-image/image' if iskaggle else 'data/isic-2024-challenge/train-image/image'

In [11]:
class CustomDataset(Dataset):
    def __init__(self, hdf5_file_path, metadata_df,target=None, transform=None):
        self.hdf5_file_path = hdf5_file_path
        self.hdf5_file = h5py.File(self.hdf5_file_path, 'r')
        self.metadata_df = metadata_df
        self.image_ids = metadata_df['isic_id']
        self.labels = target
        self.transform = transform

        self.mean_of_color_channels = None  # Initialize as None
        self.std_of_color_channels = None   # Initialize as None
        # self._calculate_stats()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids.iloc[idx]
        image = np.array(Image.open(BytesIO(np.array(self.hdf5_file[image_id]))),dtype=np.float32)/255
        
        if self.transform:
            image = self.transform(image=image)
            image = image['image']

        if self.labels is not None:
            label = self.labels.iloc[idx]
            return image, label
        else:
            return image

   

In [12]:
# HyperParameters
dim = 384 
batch_size = 64

In [13]:
train_transform = A.Compose([
    A.Resize(height=dim, width=dim), #resize 
    
    A.OneOf([
        A.Transpose(p=1.0),
        A.VerticalFlip(p=1.0),
        A.HorizontalFlip(p=1.0)
    ], p=0.5),  # Transpose, vertical flip, or horizontal flip with equal probability   

    A.GaussNoise(var_limit=(5.0, 30.0), p=1.0),  # Gaussian noise  
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=1.0),  # Hue, saturation, and value adjustment
    A.CoarseDropout(p=1.0),  # Cutout augmentation
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])


test_transform = A.Compose([
    A.Resize(height=dim, width=dim), #resize 
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

In [14]:
train_dataset = CustomDataset(train_hdf5_path,train_metadata_df,target=train_metadata_df['target'],transform=train_transform)
# train_image_dataset = CustomDatasetImage(train_image_path,train_metadata_df,target=train_metadata_df['target'],transform=train_transform)
val_dataset = CustomDataset(train_hdf5_path,val_metadata_df,target=val_metadata_df['target'],transform=train_transform)
test_dataset = CustomDataset(test_hdf5_path,test_metadata_df,transform=test_transform)

In [15]:
n_w = 0
if iskaggle:
    n_w = 4
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=n_w)
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=True,num_workers=n_w)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=n_w)

In [16]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def calculate_partial_auc_by_tpr(y_true, y_scores, min_tpr=0.8):
    v_gt = abs(y_true-1)
    v_pred = 1 - y_scores
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

In [17]:
class Model(nn.Module):
    def __init__(self,num_classes,device,dim = 32,num_epochs = 20,learning_rate = 0.001,early_stopping = False):
        super().__init__()
        self.num_of_classes = num_classes
        self.device = device
        self.dim = dim
        # Debugging
        self.DEBUG = True
        # Hyperparameters
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.early_stopping = early_stopping
        # History while Training
        self.model_loss_history = []

        self.model_train_acc_history = []
        self.model_train_pauc_history = []
        
        self.model_val_acc_history = []
        self.model_val_precision_history = []
        self.model_val_recall_history = []
        self.model_val_pauc_history = []
        
        self.model_lr_history = []

        # Model Attributes
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = None
        self.accuracy = Accuracy(task= 'binary', average='macro').to(self.device)
        self.precision = Precision(task= 'binary', average='macro').to(self.device)
        self.recall = Recall(task= 'binary', average='macro').to(self.device)
        
        # Load ResNet-18 from timm
#         self.feature_extractor = timm.create_model('resnet18', pretrained=True, num_classes=0)
        self.feature_extractor = models.resnet18(pretrained=False)
        self.feature_extractor.load_state_dict(torch.load("/kaggle/input/resnet18/resnet18.pth"))
        self.feature_extractor.to(self.device)
        

        # Add custom classifier head
        self.classifier = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256,50),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(50,1)
        ).to(self.device)
        

    def freeze_backbone(self, freeze=True):
        # Freeze/unfreeze feature extractor layers
        for param in self.feature_extractor.parameters():
            param.requires_grad = not freeze
            
    def freeze_classifier(self, freeze=True):
        for param in self.classifier.parameters():
            param.requires_grad = not freeze
    def forward(self, x):
        # Feature extraction using ResNet-18
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x
    
    def predict(self, img):
        '''
        returns the predicted classes for the given images
        '''
        self.eval()
        with torch.no_grad():
            img = img.to(self.device)
            output = self(img)
            prob_outputs = torch.sigmoid(output)
            return prob_outputs
            
        

    
    def eval_val(self, data_loader):
        '''
        returns the accuracy, precision, recall , partial AUC, predicted probabilities and actual labels for the given data loader
        '''
        self.eval()
        y_prob_pred = np.array([])
        y_actual = np.array([])
        with torch.no_grad():
            for images, actuals in data_loader:
                
                images, actuals = images.to(self.device), actuals.to(self.device)
                # Get the model predictions
                output = self(images)
                # Get the probability outputs
                prob_outputs = torch.sigmoid(output)
                # Get the binary predictions
                batch_pred = (prob_outputs > 0.5).float()
                # Reshape the actuals
                actuals = actuals.unsqueeze(1).float()
                
                y_prob_pred = np.concatenate((y_prob_pred, prob_outputs.cpu().numpy().squeeze()))
                y_actual = np.concatenate((y_actual, actuals.cpu().numpy().squeeze()))

                # Update the metrics
                self.accuracy(batch_pred, actuals)
                self.precision(batch_pred, actuals)
                self.recall(batch_pred, actuals)
        
        # Calculate the partial AUC
        partial_auc = calculate_partial_auc_by_tpr(y_true=y_actual, y_scores=y_prob_pred, min_tpr=0.8)

        return self.accuracy.compute(), self.precision.compute(), self.recall.compute(), partial_auc, y_prob_pred, y_actual
    
    def train_model(self, train_loader, val_loader):
        
        last_accuracy = -100
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        scaler = GradScaler()

        for epoch in range(self.num_epochs):
            self.train()
            running_loss = 0.0

            for i, (images, labels) in enumerate(train_loader):
                
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                with autocast():
                    outputs = self(images)
                    labels = labels.unsqueeze(1).float()
                    loss = self.criterion(outputs, labels)
                scaler.scale(loss).backward()
                # Gradient clipping
                max_norm = 1.0  # Set the maximum allowable norm for the gradients
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm)
                scaler.step(self.optimizer)
                scaler.update()

                running_loss += loss.item()
                if i%1000 == 0 and self.DEBUG:
                    print(" Step [{}/{}] Loss: {}".format(i, len(train_loader), loss.item()))
                    
            val_acc, val_precision, val_recall, val_pauc, _ , _ = self.eval_val(val_loader)
            train_acc, _, _, train_pauc, _, _ = self.eval_val(train_loader)

            self.model_loss_history.append(running_loss/len(train_loader))
            self.model_train_acc_history.append(train_acc.item())
            self.model_train_pauc_history.append(train_pauc)
            self.model_val_acc_history.append(val_acc.item())
            self.model_val_precision_history.append(val_precision.item())
            self.model_val_recall_history.append(val_recall.item())
            self.model_val_pauc_history.append(val_pauc)
            self.model_lr_history.append(self.optimizer.param_groups[0]['lr'])
            
            print(f'Epoch: {epoch+1}/{self.num_epochs}, Loss: {loss.item()},Train Acc: {train_acc}, Val Acc: {val_acc}, Val Precision: {val_precision}, Val Recall: {val_recall},Train PAUC: {train_pauc}, Val PAUC: {val_pauc}')
            
            if val_acc > last_accuracy:
                last_accuracy = val_acc
            elif self.early_stopping:
                break
        
        print('Finished Training')

    def plot_history(self):
        # making two plots one for loss and other for accuracy
        fig, axs = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Model Training History')
        axs[0, 0].plot(self.model_loss_history)
        axs[0, 0].set_title('Model Loss')
        axs[0, 0].set_xlabel('Epochs')
        axs[0, 0].set_ylabel('Loss')

        axs[0, 1].plot(self.model_train_acc_history, label='Train')
        axs[0, 1].plot(self.model_val_acc_history, label='Val')
        axs[0, 1].set_title('Model Accuracy')
        axs[0, 1].set_xlabel('Epochs')
        axs[0, 1].set_ylabel('Accuracy')
        axs[0, 1].legend()

        axs[1, 0].plot(self.model_val_precision_history)
        axs[1, 0].set_title('Model Precision')
        axs[1, 0].set_xlabel('Epochs')
        axs[1, 0].set_ylabel('Precision')
        
        axs[1, 1].plot(self.model_val_recall_history)
        axs[1, 1].set_title('Model Recall')
        axs[1, 1].set_xlabel('Epochs')
        axs[1, 1].set_ylabel('Recall')

        axs[0, 2].plot(self.model_lr_history)
        axs[0, 2].set_title('Learning Rate')
        axs[0, 2].set_xlabel('Epochs')
        axs[0, 2].set_ylabel('Learning Rate')
        
        axs[1, 2].plot(self.model_val_pauc_history)
        axs[1, 2].set_title('Model Partial AUC')
        axs[1, 2].set_xlabel('Epochs')
        axs[1, 2].set_ylabel('Partial AUC')
        

        plt.show()
    
    def save_model(self):
        torch.save(self.state_dict(),type(self).__name__+'.pth')

    def print_summary(self):
        pass
#         summary(self, (3, self.dim, self.dim))

In [18]:
cnn = Model(num_classes=2, 
            device=device, 
            dim=dim, 
            num_epochs=2, 
            learning_rate=0.01,
            early_stopping=False)
cnn.to(device)
# cnn.print_summary()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model(
  (criterion): BCEWithLogitsLoss()
  (accuracy): BinaryAccuracy()
  (precision): BinaryPrecision()
  (recall): BinaryRecall()
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_

In [19]:
cnn.num_epochs = 5
cnn.learning_rate = 0.001

In [20]:
cnn.train_model(train_loader=train_loader,val_loader=val_loader)

 Step [0/497] Loss: 0.7131290435791016
Epoch: 1/5, Loss: 0.0010769193759188056,Train Acc: 0.9900990128517151, Val Acc: 0.9900491237640381, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.020761748188846216, Val PAUC: 0.009678873965278443
 Step [0/497] Loss: 0.0025984973181039095
Epoch: 2/5, Loss: 2.50339240892572e-07,Train Acc: 0.9900990128517151, Val Acc: 0.9900906682014465, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.020315060539303584, Val PAUC: 0.0221119592875318
 Step [0/497] Loss: 0.00015256012557074428
Epoch: 3/5, Loss: 0.0,Train Acc: 0.9900990128517151, Val Acc: 0.9900944828987122, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.017020498242637322, Val PAUC: 0.01453637316899467
 Step [0/497] Loss: 5.9604616353681195e-08
Epoch: 4/5, Loss: 0.00021652717259712517,Train Acc: 0.9900990128517151, Val Acc: 0.9900959134101868, Val Precision: 0.0, Val Recall: 0.0,Train PAUC: 0.018921814145505526, Val PAUC: 0.018261088659382325
 Step [0/497] Loss: 4.472462023841217e-05
Epoch: 5

In [21]:
cnn_acc , cnn_precision, cnn_recall , cnn_pauc, y_pred, y_actual = cnn.eval_val(val_loader)
print(f"pAUC: {cnn_pauc}, Precision: {cnn_precision}, Recall: {cnn_recall}")

pAUC: 0.019828173593440763, Precision: 0.0, Recall: 0.0


In [22]:
# 0.5 threshold
y_pred = np.array(y_pred)
y_actual = np.array(y_actual).astype(int)
y_pred = (y_pred > 0.5).astype(int)

# output the incorrect predictions
incorrect_predictions = np.where(y_pred != y_actual)[0]
number_zero = np.sum(y_pred == 0)
number_one = np.sum(y_pred == 1)
number_zero, number_one

(7939, 0)

In [23]:
cnn.save_model()

In [24]:
y_prod_pred = []
for images in test_loader:
    images = images.to(device)
    prob_outputs = cnn.predict(images)
    y_prod_pred.extend(prob_outputs.cpu().numpy())

y_prod_pred = np.array(y_prod_pred)
y_prod_pred = y_prod_pred.flatten()
y_prod_pred

array([1.19176775e-08, 1.19122676e-08, 1.19087691e-08], dtype=float32)

In [25]:
submission_df = pd.DataFrame({'isic_id': test_metadata_df['isic_id'], 'target': y_prod_pred})
submission_df 

,isic_id,target
0,ISIC_0015657,1.191768e-08
1,ISIC_0015729,1.191227e-08
2,ISIC_0015740,1.190877e-08


In [26]:
submission_df.to_csv('submission.csv', index=False)